# 随机森林

### Bagging 

从数据集中随机取一个样本,然后把样本放回数据集,重复 N次,既可得 N 个样本的采样集
则: 数据集中有 63.2% 的样本出现在采样集中,剩下的 36.8% 作为验证集

### 步骤:
1. 创建随机森林
  - 使用 Bagging 选择 N 个样本
  - 随机从 M 个属性中选取 m 个属性 ($m \ll M$) 来作为分裂属性(不剪枝)
  - 按照以上 2 步建立大量决策树, 则构成随机森林
2. 使用验证集校验随机森林
3. 少于迭代次数则重复 1 


### 数据缺失的处理

1. 训练数据缺失
  - 先使用属性的均值占位
  - 创建邻近矩阵记录邻近的数据(决策树中路径一致则 +1)
  - 正规化邻近矩阵
  - 更新缺失数据 = 使用邻近矩阵中的权重 * 数据
2. 预测数据缺失
  - 先猜测结果
  - 使用属性的均值填写缺失数据
  - 分别进入随机森林预测, 分类误差少的结果胜利
